In [12]:
from fastai2.text.all import *

In [13]:
BASE = Path("./")
MODEL_PATH = BASE/'pretrained_lm'

In [14]:
# create raw text files
JSON_PATH = BASE/'dataset_dataset-Kurkuma-GoogleSERP'
TXT_PATH = BASE/'text_files'
#list(JSON_PATH.glob('*.json'))

In [31]:
import json
for file in list(JSON_PATH.glob('*.json')):
    jf = json.load(file)
    tf = TXT_PATH/(str(file.stem)+'.txt')
    tf.write_text(jf['text'])

In [4]:
get_kurkuma_files = partial(get_text_files, folders=[str(TXT_PATH)])

In [5]:
files = get_kurkuma_files(BASE)

In [38]:
print(files[0].open().read())

Skip to content

Sie haben wahrscheinlich schon einmal von Kurkuma gehört – es ist das Gewürz, das Currys und Senfgelb macht. Möglicherweise haben Sie es sogar als Ergänzung in Ihrem lokalen Naturkostladen gesehen. Dieses Kurkumapulver in Kapseln und Gewürzflaschen stammt aus einer Wurzel, die getrocknet und gemahlen wird. Eine Option, von der Sie wahrscheinlich weniger gehört haben, ist jedoch das ätherische Kurkumaöl. Kurkumaöl ist eine viel stärkere Wahl, wenn es darum geht, dieses Gewürz für verschiedene gesundheitliche Vorteile zu verwenden.

Wenn Sie das Gefühl haben, dass Ihr Nervensystem aus dem Gleichgewicht ist oder beruhigt werden muss, fügen Sie Kurkumaöl zu Kokosmilch und Honig hinzu, um ein köstliches Getränk zu erhalten.

Die wohltuenden Eigenschaften von Kurkumaöl können genutzt werden, indem man ein bis zwei Tropfen in einer Gemüsekapsel einnimmt. Wenn Sie dies regelmäßig tun, erhält Ihr Körper die antioxidative Unterstützung, die er benötigt, um sich vor Schäden durch

In [15]:
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(BASE, is_lm=True),
    get_items=get_text_files, splitter=RandomSplitter(0.1)
).dataloaders(BASE, path=BASE, bs=64, seq_len=80)

In [41]:
dls_lm.show_batch()

,text,text_
0,"xxbos xxmaj der xxmaj wirkstoff ibuprofen - lysinat bringt den betroffenen xxmaj schmerzpatienten eine längere xxmaj schmerzfreiheit . xxmaj dadurch braucht man schließlich auch weniger xxmaj tabletten , um die xxmaj schmerzen ausreichend zu lindern . xxmaj dabei gehört das ibuprofen - lysinat wie das xxmaj ibuprofen zur xxmaj gruppe der nicht - steroidalen xxmaj entzündungshemmer ( nsar ) mit den bekannten schmerzlindernden , fiebersenkenden und entzündungshemmenden xxmaj eigenschaften . xxmaj dementsprechend sind xxmaj kopfschmerzen , xxmaj migräne","xxmaj der xxmaj wirkstoff ibuprofen - lysinat bringt den betroffenen xxmaj schmerzpatienten eine längere xxmaj schmerzfreiheit . xxmaj dadurch braucht man schließlich auch weniger xxmaj tabletten , um die xxmaj schmerzen ausreichend zu lindern . xxmaj dabei gehört das ibuprofen - lysinat wie das xxmaj ibuprofen zur xxmaj gruppe der nicht - steroidalen xxmaj entzündungshemmer ( nsar ) mit den bekannten schmerzlindernden , fiebersenkenden und entzündungshemmenden xxmaj eigenschaften . xxmaj dementsprechend sind xxmaj kopfschmerzen , xxmaj migräne mit"
1,"( anzeige ) ist das xxmaj curcumin in sogenannten xxmaj mizellen eingeschlossen . \n\n xxmaj mizellen sind bipolar , das heißt auf der einen xxmaj seite wasserlöslich , auf der anderen xxmaj seite fettlöslich . \n\n xxmaj das xxmaj curcumin ist dabei innen i m fettlöslichen xxmaj teil der xxmaj mizelle eingeschlossen , gleichzeitig aber nach außen hin wasserlöslich : \n\n xxmaj der xxmaj vorteil ist , dass diese xxmaj mizellen einfach durch die xxmaj darmwand passieren können und das","anzeige ) ist das xxmaj curcumin in sogenannten xxmaj mizellen eingeschlossen . \n\n xxmaj mizellen sind bipolar , das heißt auf der einen xxmaj seite wasserlöslich , auf der anderen xxmaj seite fettlöslich . \n\n xxmaj das xxmaj curcumin ist dabei innen i m fettlöslichen xxmaj teil der xxmaj mizelle eingeschlossen , gleichzeitig aber nach außen hin wasserlöslich : \n\n xxmaj der xxmaj vorteil ist , dass diese xxmaj mizellen einfach durch die xxmaj darmwand passieren können und das xxmaj"
2,"kühlen xxmaj wasserstrahl abbrausen . \n\n xxmaj sanfte xxmaj temperaturen oder ein xxmaj aufenthalt auf der untersten xxmaj saunabank ist dafür schon ausreichend . xxmaj zwei bis drei xxmaj saunagänge und ein xxmaj saunabesuch pro xxmaj woche sind sinnvoll , um vom gesundheitlichen xxmaj nutzen zu profitieren . \n\n xxmaj auch von innen lassen sich die xxmaj abwehrkräfte steigern , natürliches xxmaj vitamin c funktioniert da besonders gut . xxmaj kleinkindern und xxmaj kindern bis 13 xxmaj jahren wird eine","xxmaj wasserstrahl abbrausen . \n\n xxmaj sanfte xxmaj temperaturen oder ein xxmaj aufenthalt auf der untersten xxmaj saunabank ist dafür schon ausreichend . xxmaj zwei bis drei xxmaj saunagänge und ein xxmaj saunabesuch pro xxmaj woche sind sinnvoll , um vom gesundheitlichen xxmaj nutzen zu profitieren . \n\n xxmaj auch von innen lassen sich die xxmaj abwehrkräfte steigern , natürliches xxmaj vitamin c funktioniert da besonders gut . xxmaj kleinkindern und xxmaj kindern bis 13 xxmaj jahren wird eine xxmaj"
3,", britische xxmaj kuchen und andere indische xxmaj süßigkeiten . \n\n xxmaj wenn du in xxmaj sri xxmaj lanka warst und keine xxmaj früchte gegessen hast , dann hast du das xxmaj beste verpasst . xxmaj kaum ein xxmaj land hat eine so reiche xxmaj auswahl an xxmaj früchten . xxmaj von vielen wirst du wahrscheinlich noch nicht gehört haben . xxmaj oder kennst du xxmaj woodapple , xxmaj xxunk oder xxmaj soursop ? xxmaj nein ? xxmaj ich kannte","britische xxmaj kuchen und andere indische xxmaj süßigkeiten . \n\n xxmaj wenn du in xxmaj sri xxmaj lanka warst und keine xxmaj früchte gegessen hast , dann hast du das xxmaj beste verpasst . xxmaj kaum ein xxmaj land hat eine so reiche xxmaj auswahl an xxmaj früchten . xxmaj von vielen wirst du wahrscheinlich noch nicht gehört haben . xxmaj oder kennst du xxmaj woodapple , xxmaj xxunk oder xxmaj so

In [16]:
FILE_LM_ENCODER = MODEL_PATH/'30k-pre-ger'
FILE_ITOS = MODEL_PATH/'30k-pre-ger-itos'
config = awd_lstm_lm_config.copy()
config['n_hid'] = 1150

In [17]:
learn = language_model_learner(dls_lm, 
                               AWD_LSTM, 
                               config=config, 
                               pretrained_fnames=[FILE_LM_ENCODER, FILE_ITOS],
                               drop_mult=0.3,
                               metrics=[accuracy, Perplexity()]).to_fp16()

In [9]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.267177,2.959113,0.437075,19.280869,1:05:39


In [10]:
learn.save('1epoch')

In [12]:
learn.freeze_to(-1)

In [13]:
learn.fit_one_cycle(6, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.171355,2.931623,0.436947,18.758049,1:05:46
1,3.133454,2.862837,0.441940,17.511141,1:05:17
2,2.948828,2.715784,0.461219,15.116462,1:05:29
3,2.755808,2.549217,0.486193,12.797079,1:05:12
4,2.633215,2.421087,0.509760,11.258085,1:05:30


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
learn.save('1epoch-freeze-1-6epochs')

In [18]:
learn.load('1epoch-freeze-1-6epochs')

In [19]:
learn.freeze_to(-2)

In [21]:
learn.fit_one_cycle(6, slice(1e-5, 1e-3))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.565369,2.198321,0.539743,9.009874,1:05:56
1,2.474988,2.167107,0.543921,8.732980,1:06:28
2,2.485996,2.144728,0.547554,8.539720,1:06:47
3,2.409091,2.129349,0.550402,8.409387,1:07:48
4,2.417698,2.122807,0.551509,8.354558,1:06:08
5,2.392699,2.121427,0.551867,8.343035,1:06:34


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
learn.save('1epoch-freeze-2-6epochs')

In [23]:
TEXT = "Kurkuma ist für die Gesundheit wichtig, weil"
N_WORDS = 400
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [11]:
print("\n".join(preds))

Kurkuma ist für die Gesundheit wichtig , weil es in vielen Fällen nur stark an Curry erinnert . Denn für viele Menschen ist die Einnahme dieser Ernährung völlig ungefährlich . Die Kurkuma ist auch eine gesunde Alternative zur Kurkuma . Wenn man viel Kurkuma Pulver oder Kurkuma zu sich nimmt , kann man man auch sehr viel Kurkuma aufnehmen . Die Kurkuma hat eine entzündungshemmende Wirkung , und sie wird sogar in einer Kapsel verkauft . Die Gelbwurzel ist sogar ein natürliches Multitalent .   Kurkuma ( curcuma longa ) wird unterschiedliche Wirkungen zugesprochen , wobei es nicht nur eine entzündungshemmende Wirkung hat , sondern auch eine entzündungshemmende Wirkung hat . 

 Curcumin gehört zur Familie der Ingwergewächse und stammt aus Südasien . Die Wurzel wird traditionell als Gewürz in der traditionellen chinesischen und indischen Medizin verwendet . In der fernöstlichen Lehre wird Kurkuma auch als Heilmittel gegen Verdauungsbeschwerden verwendet . Es wird in der Ayurveda - medizin zu